In [28]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import KBinsDiscretizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score, classification_report

In [2]:
df = pd.read_csv('credit_dataset.csv')

/var/folders/30/bqhvgy251t71zrd5hbqbfxlc0000gn/T/ipykernel_71026/566232830.py:1: DtypeWarning: Columns (20) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('credit_dataset.csv')


In [12]:
df.loc[:, ['id', 'loan_status', ]].groupby(['loan_status']).count()

,id
loan_status,
Charged Off,42475
Current,224226
Default,832
Does not meet the credit policy. Status:Charged Off,761
Does not meet the credit policy. Status:Fully Paid,1988
Fully Paid,184739
In Grace Period,3146
Late (16-30 days),1218
Late (31-120 days),6900


In [13]:
def map_loan_status(status):
    if status == 'Fully Paid':
        return 0
    elif status in ['Default', 'In Grace Period', 'Late (16-30 days)', 'Late (31-120 days)']:
        return 1
    return np.nan

df['PD'] = df['loan_status'].apply(map_loan_status)
df = df.dropna(subset=['PD'])

In [14]:
df.head()

,Unnamed: 0,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,...,il_util,open_rv_12m,open_rv_24m,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,inq_last_12m,PD
0,0,1077501,1296599,5000,5000,4975.0,36 months,10.65,162.87,B,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
2,2,1077175,1313524,2400,2400,2400.0,36 months,15.96,84.33,C,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
3,3,1076863,1277178,10000,10000,10000.0,36 months,13.49,339.31,C,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
5,5,1075269,1311441,5000,5000,5000.0,36 months,7.90,156.46,A,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
7,7,1072053,1288686,3000,3000,3000.0,36 months,18.64,109.43,E,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0


In [17]:
df['has_delinq_record'] = df['mths_since_last_delinq'].notnull().astype(int)
df['mths_since_last_delinq'].fillna(0, inplace=True)
df['mths_since_last_record'].fillna(0, inplace=True)

/var/folders/30/bqhvgy251t71zrd5hbqbfxlc0000gn/T/ipykernel_71026/63084121.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['has_delinq_record'] = df['mths_since_last_delinq'].notnull().astype(int)
/var/folders/30/bqhvgy251t71zrd5hbqbfxlc0000gn/T/ipykernel_71026/63084121.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the opera

In [18]:
numeric_cols = df.select_dtypes(include=['float64', 'int64']).columns
categorical_cols = df.select_dtypes(include=['object']).columns
for col in numeric_cols:
    df[col].fillna(df[col].median(), inplace=True)
for col in categorical_cols:
    df[col].fillna(df[col].mode()[0], inplace=True)

/var/folders/30/bqhvgy251t71zrd5hbqbfxlc0000gn/T/ipykernel_71026/1920472642.py:4: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[col].fillna(df[col].median(), inplace=True)
/var/folders/30/bqhvgy251t71zrd5hbqbfxlc0000gn/T/ipykernel_71026/1920472642.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col].fillna(df[col].median(), inplace=T

In [20]:
df['annual_inc'] = df['annual_inc'].clip(upper=df['annual_inc'].quantile(0.99))
df['dti'] = df['dti'].clip(upper=100)

/var/folders/30/bqhvgy251t71zrd5hbqbfxlc0000gn/T/ipykernel_71026/2742121800.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['annual_inc'] = df['annual_inc'].clip(upper=df['annual_inc'].quantile(0.99))
/var/folders/30/bqhvgy251t71zrd5hbqbfxlc0000gn/T/ipykernel_71026/2742121800.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['dti'] = df['dti'].clip(upper=100)


In [21]:
drop_cols = ['Unnamed: 0', 'id', 'member_id', 'url', 'desc', 'title', 'emp_title', 'zip_code', 'addr_state']
df = df.drop(columns=drop_cols)

In [22]:
continuous_cols = ['loan_amnt', 'int_rate', 'annual_inc', 'dti', 'revol_util']
categorical_features = ['grade', 'home_ownership', 'verification_status', 'purpose', 'term']
features = continuous_cols + categorical_features

In [23]:
for col in continuous_cols:
    discretizer = KBinsDiscretizer(n_bins=5, encode='ordinal', strategy='quantile')
    df[f'{col}_binned'] = discretizer.fit_transform(df[[col]])

In [24]:
def calculate_woe_iv(df, feature, target):
    df_woe = df.groupby(feature)[target].agg(['count', 'sum']).rename(columns={'sum': 'bads', 'count': 'total'})
    df_woe['goods'] = df_woe['total'] - df_woe['bads']
    df_woe['goods'] = df_woe['goods'].replace(0, 0.5)
    df_woe['bads'] = df_woe['bads'].replace(0, 0.5)
    df_woe['p_good'] = df_woe['goods'] / df_woe['goods'].sum()
    df_woe['p_bad'] = df_woe['bads'] / df_woe['bads'].sum()
    df_woe['woe'] = np.log(df_woe['p_good'] / df_woe['p_bad'])
    df_woe['iv'] = (df_woe['p_good'] - df_woe['p_bad']) * df_woe['woe']
    iv = df_woe['iv'].sum()
    return df_woe, iv

In [25]:
iv_dict = {}
woe_dict = {}
for feature in features:
    if feature in continuous_cols:
        df_woe, iv = calculate_woe_iv(df, f'{feature}_binned', 'PD')
    else:
        df_woe, iv = calculate_woe_iv(df, feature, 'PD')
    iv_dict[feature] = iv
    woe_dict[feature] = df_woe['woe']

# Transform features to WoE
for feature in features:
    if feature in continuous_cols:
        df[f'{feature}_woe'] = df[f'{feature}_binned'].map(woe_dict[feature])
    else:
        df[f'{feature}_woe'] = df[feature].map(woe_dict[feature])

In [26]:
selected_features = [f for f in features if iv_dict[f] > 0.02]
woe_cols = [f'{f}_woe' for f in selected_features]
print("Selected Features based on IV > 0.02:", selected_features)

Selected Features based on IV > 0.02: ['loan_amnt', 'int_rate', 'dti', 'revol_util', 'grade', 'verification_status', 'purpose', 'term']


In [29]:
# Step 6: Train-Test-Validation Split
X = df[woe_cols]
y = df['PD']
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, random_state=1, stratify=y)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=1, stratify=y_temp)

In [31]:
model = LogisticRegression(class_weight='balanced', random_state=1)
model.fit(X_train, y_train)

LogisticRegression(class_weight='balanced', random_state=1)

In [32]:
y_val_pred = model.predict(X_val)
y_val_pred_proba = model.predict_proba(X_val)[:, 1]
print("\nValidation Classification Report:\n", classification_report(y_val, y_val_pred))
print("Validation ROC AUC:", roc_auc_score(y_val, y_val_pred_proba))


Validation Classification Report:
               precision    recall  f1-score   support

         0.0       0.97      0.65      0.77     27711
         1.0       0.11      0.65      0.18      1814

    accuracy                           0.65     29525
   macro avg       0.54      0.65      0.48     29525
weighted avg       0.91      0.65      0.74     29525

Validation ROC AUC: 0.7045184771931525


In [33]:
y_test_pred = model.predict(X_test)
y_test_pred_proba = model.predict_proba(X_test)[:, 1]
print("\nTest Classification Report:\n", classification_report(y_test, y_test_pred))
print("Test ROC AUC:", roc_auc_score(y_test, y_test_pred_proba))


Test Classification Report:
               precision    recall  f1-score   support

         0.0       0.97      0.64      0.77     27711
         1.0       0.11      0.68      0.19      1815

    accuracy                           0.64     29526
   macro avg       0.54      0.66      0.48     29526
weighted avg       0.92      0.64      0.73     29526

Test ROC AUC: 0.7128687149030236


In [34]:
PDO = 20
Factor = PDO / np.log(2)  # ~28.85
Offset = 700 + Factor * np.log(20)  # ~784

# Calculate scores on test set
odds = (1 - y_test_pred_proba) / y_test_pred_proba  # Good-to-bad odds
scores = Offset + Factor * np.log(odds)

# Display sample scores
score_df = pd.DataFrame({
    'Probability_of_Default': y_test_pred_proba,
    'Odds': odds,
    'Score': scores
})
print("\nSample Scores:\n", score_df.head())


Sample Scores:
    Probability_of_Default      Odds       Score
0                0.182028  4.493669  829.796437
1                0.348789  1.867059  804.453914
2                0.408214  1.449695  797.153542
3                0.263208  2.799276  816.139632
4                0.560165  0.785187  779.460739


In [35]:
coef_dict = dict(zip(woe_cols, model.coef_[0]))
points_dict = {}
for feature in selected_features:
    feature_woe = woe_dict[feature]
    coef = coef_dict[f'{feature}_woe']
    points = -Factor * coef * feature_woe
    points_dict[feature] = points

In [36]:
print("\nPoints Allocation per Feature Bin:")
for feature, points in points_dict.items():
    print(f"\n{feature}:\n", points)


Points Allocation per Feature Bin:

loan_amnt:
 loan_amnt_binned
0.0    6.207283
1.0    2.715569
2.0   -0.182026
3.0   -1.660815
4.0   -4.018355
Name: woe, dtype: float64

int_rate:
 int_rate_binned
0.0   -13.607663
1.0    -5.922410
2.0    -1.827242
3.0     2.809584
4.0     8.039133
Name: woe, dtype: float64

dti:
 dti_binned
0.0    10.406010
1.0     6.043349
2.0     2.299711
3.0    -1.888195
4.0   -11.075065
Name: woe, dtype: float64

revol_util:
 revol_util_binned
0.0    3.208460
1.0    0.395835
2.0   -0.529320
3.0   -0.725891
4.0   -1.367992
Name: woe, dtype: float64

grade:
 grade
A    45.868075
B    16.175654
C    -5.787627
D   -17.496216
E   -29.962995
F   -31.807380
G   -38.527632
Name: woe, dtype: float64

verification_status:
 verification_status
Not Verified       5.531015
Source Verified   -4.110374
Verified          -0.799658
Name: woe, dtype: float64

purpose:
 purpose
car                   19.834604
credit_card            0.929487
debt_consolidation    -1.795618
educatio